In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
from google.cloud import storage
from google.cloud.storage.bucket import Bucket

from initial import parse_company_page, post_process_company_df, parse_round_page, post_process_round_df, parse_investor_page, post_process_investor_df, parse_acquisition_page, post_process_acquisition_df, map_company_status
import pandas as pd
from tqdm.notebook import tqdm
from utils import get_base_domain, cleanup_prefectur_info, get_usd, split2row_with_index, json_col_to_df
from cb import generate_cb_uuid
from datetime import datetime

pd.options.display.max_columns = 999

In [5]:

client = storage.Client()
bucket: Bucket = client.get_bucket("initial-htmls")

# Parse data from GCS
## Parse Company Page

In [6]:
blobs = bucket.list_blobs(prefix="initial.inc/companies")
blob_cnt = sum(1 for _ in blobs)

company_df_list = []
blobs = bucket.list_blobs(prefix="initial.inc/companies")
for blob in tqdm(blobs, total=blob_cnt):
    html_byte = blob.download_as_string()
    df = parse_company_page(html_byte.decode('utf-8'))
    # TODO. need to add timestamp and remove duplicates by lateste timestamp
    company_df_list.append(df)

company_df = pd.concat(company_df_list, axis=0)
print(len(company_df))
company_df.head()

  0%|          | 0/408 [00:00<?, ?it/s]

20316


,Company name,Description,tag,INITIAL series,status,status update date,Assignee,memo,Representative's name,Total Funding Amount.(thousand yen),Total Procurement Calculation Date,Post-Money Valuation.(thousand yen),Valuation calculation round implementation date,Prefectures,Founded Date,Last Funding Date,type,registration date,update date,Investigation status,Market capitalization at the time of IPO (initial price).(One million yen),IPO date,IPO classification,number of employees,Shareholder status,origin,Universities and research institutes,Industry,market,address,Corporate number,phone number,website
0,"[{'name': 'Notus Solar Japan Co., Ltd.', 'url'...","[Using the solar sharing ""Notus Solar System"" ...",[],[],[],[],[],[],[Ryuzo Takahashi],[N/A],[],[],[],[Osaka],[2022/04/01],[2023/03/15],[private company],[2023/03/15],[2023/03/15],[],[],[],[],[],[],[],[],[Environment related],[],"[Room 1105, NLC Central Building, 4-3-21 Nishi...",[5120001245305],[],[https://notussolarjapan.co.jp/]
1,"[{'name': 'Helte Co., Ltd.', 'url': '/companie...","[Providing “Sail”, a global communication serv...","[#edtech, #study, #online-service, #btoc]",[Series A],[],[],[],[],[Manabu Goto],"[226,548]",[2021/06/22],"[525,256]",[2021/05/19],[Chiba prefecture],[2016/03/22],[2023/03/01],[private company],[2018/11/02],[2023/03/15],[Under Investigation],[],[],[],[17],"[Uncertain about Investors, with Invested by c...",[],[],[Computers - IT Services],[],"[First Mitoya Building 3F, 2-28 Higashigamicho...",[2040001094614],[050-3358-6892],[https://www.helte.jp/]
2,"[{'name': 'Posiwill Co., Ltd.', 'url': '/compa...",[Developing business centered on “POSIWILL CAR...,"[#human-development, #online-service, #career-...",[Series A],[],[],[],[],[Mei Kanai],"[219,844]",[2020/12/02],"[1,000,862]",[2020/08/03],[Tokyo],[2017/08/21],[2020/08/03],[private company],[2018/09/27],[2023/03/15],[Under Investigation],[],[],[],[42],[Invested by VCs],[],[],[Consumer services and sales],[],"[Tokyo Tatemono Aoyama Building 8F, 3-3-5 Kita...",[6010401133425],[],[https://www.posiwill.co.jp/]
3,"[{'name': 'oVice Co., Ltd.', 'url': '/companie...",[Developed and provided a virtual space “oVice...,"[#organization, #hrtech, #cloud-services, #sof...",[Series B],[],[],[],[],[Ding Seho],"[6,049,286]",[2022/09/14],"[19,906,049]",[2022/08/26],[Ishikawa Prefecture],[2020/02/20],[2022/08/26],[private company],[2020/10/19],[2023/03/15],[Under Investigation],[],[],[],[125],[Invested by VCs],[],[],[business service],[],"[113 B, Honfuchu-cho, Nanao City, Ishikawa Pre...",[9010401151283],[],[https://ovice.in/]
4,"[{'name': 'Things Inc.', 'url': '/companies/A-...",[Developed and operated cloud-based product de...,"[#cloud-services, #software, #btob, #hardware,...",[seed],[],[],[],[],[Atsuya Suzuki],"[259,006]",[2023/03/15],"[640,005]",[2023/02/07],[Tokyo],[2021/09/21],[2023/03/15],[private company],[2021/12/03],[2023/03/15],[Under Investigation],[],[],[],[],[Invested by VCs],[],[],[Computers - IT Services],[],"[5-25-18 Hongo, Bunkyo-ku, Tokyo Hi-Tech Hongo...",[1010001221329],[090-9392-1541],[https://things-inc.com/]


In [7]:
company_df['Company name'].apply(lambda x: len(x)).value_counts()

1    20316
Name: Company name, dtype: int64

In [8]:
company_df = post_process_company_df(company_df)
print(len(company_df))
company_df.head()

19519


,Company name,Description,tag,INITIAL series,status,status update date,Assignee,memo,Representative's name,Total Funding Amount.(thousand yen),Total Procurement Calculation Date,Post-Money Valuation.(thousand yen),Valuation calculation round implementation date,Prefectures,Founded Date,Last Funding Date,type,registration date,update date,Investigation status,Market capitalization at the time of IPO (initial price).(One million yen),IPO date,IPO classification,number of employees,Shareholder status,origin,Universities and research institutes,Industry,market,address,Corporate number,phone number,website,Company url
0,"Notus Solar Japan Co., Ltd.","Using the solar sharing ""Notus Solar System"" d...",[],,,,,,Ryuzo Takahashi,N/A,,[],,Osaka,2022/04/01,2023/03/15,private company,2023/03/15,2023/03/15,,,,,,,,,Environment related,,"Room 1105, NLC Central Building, 4-3-21 Nishin...",5120001245305,,https://notussolarjapan.co.jp/,/companies/A-45746
1,"Helte Co., Ltd.","Providing “Sail”, a global communication servi...","[#edtech, #study, #online-service, #btoc]",Series A,,,,,Manabu Goto,"226,548",2021/06/22,"[525,256]",2021/05/19,Chiba prefecture,2016/03/22,2023/03/01,private company,2018/11/02,2023/03/15,Under Investigation,,,,17,"Uncertain about Investors, with Invested by co...",,,Computers - IT Services,,"First Mitoya Building 3F, 2-28 Higashigamicho,...",2040001094614,050-3358-6892,https://www.helte.jp/,/companies/A-31747
2,"Posiwill Co., Ltd.",Developing business centered on “POSIWILL CARE...,"[#human-development, #online-service, #career-...",Series A,,,,,Mei Kanai,"219,844",2020/12/02,"[1,000,862]",2020/08/03,Tokyo,2017/08/21,2020/08/03,private company,2018/09/27,2023/03/15,Under Investigation,,,,42,Invested by VCs,,,Consumer services and sales,,"Tokyo Tatemono Aoyama Building 8F, 3-3-5 Kita-...",6010401133425,,https://www.posiwill.co.jp/,/companies/A-31478
3,"oVice Co., Ltd.",Developed and provided a virtual space “oVice”...,"[#organization, #hrtech, #cloud-services, #sof...",Series B,,,,,Ding Seho,"6,049,286",2022/09/14,"[19,906,049]",2022/08/26,Ishikawa Prefecture,2020/02/20,2022/08/26,private company,2020/10/19,2023/03/15,Under Investigation,,,,125,Invested by VCs,,,business service,,"113 B, Honfuchu-cho, Nanao City, Ishikawa Pref...",9010401151283,,https://ovice.in/,/companies/A-37652
4,Things Inc.,Developed and operated cloud-based product dev...,"[#cloud-services, #software, #btob, #hardware,...",seed,,,,,Atsuya Suzuki,"259,006",2023/03/15,"[640,005]",2023/02/07,Tokyo,2021/09/21,2023/03/15,private company,2021/12/03,2023/03/15,Under Investigation,,,,,Invested by VCs,,,Computers - IT Services,,"5-25-18 Hongo, Bunkyo-ku, Tokyo Hi-Tech Hongo ...",1010001221329,090-9392-1541,https://things-inc.com/,/companies/A-40961


## Parse funding round page

In [9]:
blobs = bucket.list_blobs(prefix="initial.inc/rounds")
blob_cnt = sum(1 for _ in blobs)

blobs = bucket.list_blobs(prefix="initial.inc/rounds")
rounds_df_list = []

for blob in tqdm(blobs, total=blob_cnt):
    html_byte = blob.download_as_string()
    df = parse_round_page(html_byte.decode('utf-8'))
    # TODO. need to add timestamp and remove duplicates by lateste timestamp
    rounds_df_list.append(df)

rounds_df = pd.concat(rounds_df_list, axis=0).reset_index(drop=True)
print(len(rounds_df))
rounds_df.head()

  0%|          | 0/1144 [00:00<?, ?it/s]

57157


,,Procurement date,Company Name,INITIAL series,Funding Amount.(thousand yen),"Pre-Money Valuation.(1,000 yen)",Post-Money Valuation .(thousand yen),Investors
0,[detail],[2023/04/01],"[{'name': 'Wellmo Co., Ltd.', 'url': '/compani...",[],[],[],[],[]
1,[detail],[2023/03/24],"[{'name': 'Yura Holdings Co., Ltd.', 'url': '/...",[],"[16,704]",[],[],"[{'name': 'FUNDINNO', 'url': '/investors/V1023..."
2,[detail],[2023/03/17],"[{'name': 'Editor Camp Co., Ltd.', 'url': '/co...",[],"[70,000]",[],[],"[{'name': 'East Ventures', 'url': '/investors/..."
3,[detail],[2023/03/16],"[{'name': '24karat Inc.', 'url': '/companies/A...",[Seed],"[20,000]",[],[],[]
4,[detail],[2023/03/16],"[{'name': 'AI Communis Pte.', 'url': '/compani...",[],[],[],[],"[{'name': 'Japanesepoint', 'url': '/investors/..."


In [10]:
rounds_df = post_process_round_df(rounds_df)
print(len(rounds_df))
rounds_df.head()

57157


,Procurement date,Company Name,INITIAL series,Funding Amount.(thousand yen),"Pre-Money Valuation.(1,000 yen)",Post-Money Valuation .(thousand yen),Investors,Round Url,Company Url
0,2023/04/01,"Wellmo Co., Ltd.",,,,,[],/companies/A-15981/funding_rounds/A-15981-58,/companies/A-15981
1,2023/03/24,"Yura Holdings Co., Ltd.",,"16,704",,,"[{'name': 'FUNDINNO', 'url': '/investors/V1023...",/companies/A-40656/funding_rounds/A-40656-04,/companies/A-40656
2,2023/03/17,"Editor Camp Co., Ltd.",,"70,000",,,"[{'name': 'East Ventures', 'url': '/investors/...",/companies/A-45806/funding_rounds/A-45806-01,/companies/A-45806
3,2023/03/16,24karat Inc.,Seed,"20,000",,,[],/companies/A-40853/funding_rounds/A-40853-04,/companies/A-40853
4,2023/03/16,AI Communis Pte.,,,,,"[{'name': 'Japanesepoint', 'url': '/investors/...",/companies/A-40256/funding_rounds/A-40256-05,/companies/A-40256


## Parse investors page

In [11]:
blobs = bucket.list_blobs(prefix="initial.inc/investors")
blob_cnt = sum(1 for _ in blobs)

blobs = bucket.list_blobs(prefix="initial.inc/investors")
investor_df_list = []

for blob in tqdm(blobs, total=blob_cnt):
    html_byte = blob.download_as_string()
    df = parse_investor_page(html_byte.decode('utf-8'))
    # TODO. need to add timestamp and remove duplicates by lateste timestamp
    investor_df_list.append(df)

investor_df = pd.concat(investor_df_list, axis=0).reset_index(drop=True)
print(len(investor_df))
investor_df.head()

  0%|          | 0/20 [00:00<?, ?it/s]

1000


,Investors,Type,Industry,
0,"[{'name': 'Mercuria Holdings Co., Ltd.', 'url'...",[VCs],[Independent],[]
1,"[{'name': 'Saitama Institute of', 'url': '/inv...",[Local governments/universities/foundations/ot...,[university],[]
2,[{'name': 'Yamaguchi Gakuen ECC Computer Colle...,[Local governments/universities/foundations/ot...,[Others],[]
3,[{'name': 'LOCAL-Z Limited Liability Partnersh...,[VCs],[Independent],[]
4,"[{'name': 'Yoii', 'url': '/investors/V19351'}]",[business corporation],[Finance/real estate],[]


In [12]:
investor_df = post_process_investor_df(investor_df)
print(len(investor_df))
investor_df.head()

1000


,Type,Industry,Investor Url,Investor Name
0,VCs,Independent,/investors/V19355,"Mercuria Holdings Co., Ltd."
1,Local governments/universities/foundations/oth...,university,/investors/V19354,Saitama Institute of
2,Local governments/universities/foundations/oth...,Others,/investors/V19353,Yamaguchi Gakuen ECC Computer College
3,VCs,Independent,/investors/V19352,LOCAL-Z Limited Liability Partnership
4,business corporation,Finance/real estate,/investors/V19351,Yoii


# Parse aquisition page

In [13]:
blobs = bucket.list_blobs(prefix="initial.inc/finance_news")
aquisition_dicts = []
html_type = ""

for blob in blobs:
    html_byte = blob.download_as_string()
    my_dict = parse_acquisition_page(html_byte.decode('utf-8'))
    # TODO. need to add timestamp and remove duplicates by lateste timestamp
    aquisition_dicts.append(my_dict)
    
acquisition_df = pd.DataFrame(aquisition_dicts)
print(len(acquisition_df))
acquisition_df.head()

34


,Startup,Industry,Founded Date,Description,Date,Acquisition amount,acquirer,News URL,Procurement amount,Underwriter
0,"[{'name': '株式会社マリンナノファイバー', 'url': '/companies...",[バイオテクノロジー],[2016/04/28],[カニ殻由来の新素材「マリンナノファイバー」に関する製品を研究開発・製造販売。\nマリンナノ...,[2023/01/11],[金額不明],"[{'name': '星光PMC株式会社', 'url': '/investors/V187...",[https://www.seikopmc.co.jp/wp-content/uploads...,NaN,NaN
1,"[{'name': '株式会社オシエテ', 'url': '/companies/A-342...",[コンピューター - ITサービス],[2018/09/13],[法人向けオンライン通訳・翻訳サービス「OCiETe（オシエテ）」を提供。\n企業と通訳者の...,[2022/12/09],"[600,000千円]","[{'name': 'CRGホールディングス株式会社', 'url': '/investor...",[https://contents.xj-storage.jp/xcontents/AS04...,NaN,NaN
2,"[{'name': '株式会社サクラエクスチェンジビットコイン', 'url': '/com...",[金融・保険・不動産],[2017/08/21],[暗号資産取次所「SEBC」にてビットコインの運用関連サービスの展開。\nSEBCは、暗号資...,[2022/11/30],[金額不明],"[{'name': 'Binance (AP) Holdings Limited', 'ur...",[https://sebc.co.jp/22113002_notice/],NaN,NaN
3,"[{'name': '株式会社インフォキュービック・ジャパン', 'url': '/comp...",[ビジネスサービス],[2007/01/01],[海外デジタルマーケティング、海外SNSマーケティング、海外ウェブクリエイティブ事業等を展開],[2022/12/01],[金額不明],"[{'name': 'デジタル・アドバタイジング・コンソーシアム株式会社', 'url': ...",[https://www.dac.co.jp/index.php?p=press/2022/...,NaN,NaN
4,"[{'name': '株式会社エアーズ', 'url': '/companies/A-396...",[ビジネスサービス],[2019/09/02],[ドローン関連事業の展開。\nJUAVACドローンエキスパートアカデミーと連携し、ドローンパ...,[2022/12/12],"[200,000千円]","[{'name': 'ジョルダン株式会社', 'url': '/investors/V047...",[https://www.jorudan.co.jp/ir/data/ir/pdf_ir/2...,NaN,NaN


In [14]:
acquisition_df = post_process_acquisition_df(acquisition_df)

# Transfer data to CB format
## First clean up company's prefecture table

In [15]:
company_df['Prefectures'] = cleanup_prefectur_info(company_df['Prefectures'])
company_prefectur_map = company_df.set_index('Company url')['Prefectures']

## Get CB Funding Round

In [16]:
fr_cols = ['uuid', 'name', 'type', 'permalink', 'cb_url', 'rank', 'created_at',
       'updated_at', 'country_code', 'state_code', 'region', 'city',
       'investment_type', 'announced_on', 'raised_amount_usd', 'raised_amount',
       'raised_amount_currency_code', 'post_money_valuation_usd',
       'post_money_valuation', 'post_money_valuation_currency_code',
       'investor_count', 'org_uuid', 'org_name', 'lead_investor_uuids']

for date_col in ['Procurement date']:
       rounds_df[date_col] = rounds_df[date_col].apply(lambda x: datetime.strptime(x, "%Y/%m/%d").strftime("%Y-%m-%d"))

rounds_df['uuid'] = rounds_df['Round Url'].apply(lambda x: generate_cb_uuid("funding_rounds", x))
rounds_df['name'] = rounds_df.apply(lambda row: f"{row['INITIAL series']} - {row['Company Name']}", axis=1)
rounds_df['type'] = 'funding_round'
rounds_df.rename(columns={'Round Url': 'permalink'}, inplace=True)
rounds_df['cb_url'] = ''
rounds_df['rank'] = ''
rounds_df['created_at'] = rounds_df['Procurement date']
rounds_df['updated_at'] = rounds_df['Procurement date']
rounds_df['country_code'] = 'JP'
rounds_df['state_code'] = ''
rounds_df['region'] = rounds_df['Company Url'].map(company_prefectur_map)
rounds_df['city'] = ''
rounds_df['investment_type'] = rounds_df['INITIAL series'].apply(lambda x: x.lower().replace(" ", "_"))
rounds_df.rename(columns={'Procurement date': 'announced_on'}, inplace=True)

rounds_df['raised_amount'] = rounds_df['Funding Amount.(thousand yen)'].apply((lambda x: float(x.replace(",", "")) * 1e3 if x else 0))
rounds_df['raised_amount_currency_code'] = 'JPY'
rounds_df['raised_amount_usd'] = rounds_df.apply(lambda row: get_usd(row['raised_amount_currency_code'], row['announced_on'], row['raised_amount']), axis=1)
rounds_df['post_money_valuation'] = rounds_df['Pre-Money Valuation.(1,000 yen)'].apply((lambda x: float(x.replace(",", "")) * 1e3 if x else 0))
rounds_df['post_money_valuation_currency_code'] = 'JPY'
rounds_df['post_money_valuation_usd'] = rounds_df.apply(lambda row: get_usd(row['post_money_valuation_currency_code'], row['announced_on'], row['post_money_valuation']), axis=1)

rounds_df['investor_count'] = rounds_df['Investors'].apply(len)
rounds_df['org_uuid'] = rounds_df['Company Url'].apply(lambda x: generate_cb_uuid("organization", x))
rounds_df.rename(columns={'Company Name': 'org_name'}, inplace=True)

rounds_df['lead_investor_uuids'] = rounds_df['Investors'].apply(lambda x: generate_cb_uuid("organizations", x[0]['url']) if x else '')

cb_rounds_df = rounds_df[fr_cols]

In [ ]:
# get funding rounds url that need to be crawled

## Test Funding Round data

In [17]:
assert (rounds_df['created_at'] == '').sum() == 0, 'missing created_at value'
assert (rounds_df['announced_on'] == '').sum() == 0, 'missing announced_on value'
assert rounds_df['uuid'].nunique() == len(rounds_df), 'It seems like the uuid is not unique across the table'


## Invesment.csv
True investment dollar value is ignored at this time.

In [18]:
investments_df = json_col_to_df(split2row_with_index(rounds_df[['Investors', 'name', 'permalink', 'uuid', 'announced_on']], 'Investors', show_index=True).reset_index(drop=True), col_name='Investors', prefix=True)
investments_df = investments_df.drop_duplicates(['permalink', 'Investors_url', 'announced_on'])
investments_df['Investors_url'].fillna('', inplace=True)
investments_df['Investors_name'].fillna('', inplace=True)

/Users/iansu/Dropbox/Mac (3)/Documents/Initial/notebooks/utils.py:151: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  tmp = df[colname].apply(pd.Series, 1).stack().reset_index()


In [19]:
inv_cols = ['uuid', 'name', 'type', 'permalink', 'cb_url', 'rank', 'created_at',
       'updated_at', 'funding_round_uuid', 'funding_round_name',
       'investor_uuid', 'investor_name', 'investor_type', 'is_lead_investor']


investments_df.rename(columns={'uuid': 'funding_round_uuid'}, inplace=True)
investments_df.rename(columns={'name': 'funding_round_name'}, inplace=True)
investments_df['uuid'] = investments_df.apply(lambda row: generate_cb_uuid("investments", row['permalink'] + row['Investors_url']), axis=1)

investments_df['name'] = investments_df.apply(lambda row: f"{row['Investors_name']} in {row['funding_round_name']}", axis=1)
investments_df['type'] = 'investment'
investments_df['permalink'] = ''
investments_df['cb_url'] = ''
investments_df['rank'] = ''
investments_df['created_at'] = investments_df['announced_on']
investments_df['updated_at'] = investments_df['announced_on']
investments_df['investor_uuid'] = investments_df['Investors_url'].apply(lambda x: generate_cb_uuid("organizations", x))
investments_df.rename(columns={'Investors_name': 'investor_name'}, inplace=True)
investments_df['investor_type'] = 'organization'
investments_df['is_lead_investor'] = (investments_df['list_index'] == 0)

cb_investments_df = investments_df[inv_cols]

In [ ]:
# get investments data that need to crawl

In [46]:
cb_investments_df.head()

,uuid,name,type,permalink,cb_url,rank,created_at,updated_at,funding_round_uuid,funding_round_name,investor_uuid,investor_name,investor_type,is_lead_investor
0,58dfd788-d2fe-aac7-ac75-18992bb1257a,"in - Wellmo Co., Ltd.",investment,,,,2023-04-01,2023-04-01,03d051a6-788a-61f9-1708-735ecc40145f,"- Wellmo Co., Ltd.",314b3d39-9fdb-6fdd-4bbe-56d4e9a0805a,,organization,False
1,f0ee5328-a066-9a89-1e6d-944f351e4cff,"FUNDINNO in - Yura Holdings Co., Ltd.",investment,,,,2023-03-24,2023-03-24,2169d092-8212-9682-e18d-8597acb8b9fe,"- Yura Holdings Co., Ltd.",b0d971d8-d590-fe9e-182b-87cd7bd3cfa3,FUNDINNO,organization,True
2,4e483c58-6bc7-9c7a-19d4-8ea074fcec8c,"East Ventures in - Editor Camp Co., Ltd.",investment,,,,2023-03-17,2023-03-17,5d84e5cf-f340-4096-09fa-88cb577ee249,"- Editor Camp Co., Ltd.",0394c337-55bb-7fe7-84a4-e8c4d8ea9c03,East Ventures,organization,True
3,3e63c965-cb82-8595-4ebb-980ca8e18e90,"Angel Investor in - Editor Camp Co., Ltd.",investment,,,,2023-03-17,2023-03-17,5d84e5cf-f340-4096-09fa-88cb577ee249,"- Editor Camp Co., Ltd.",bf8165d6-fb91-2994-c281-88a4b49eed90,Angel Investor,organization,False
4,4f435a4a-c1b9-1d4a-87ce-98a0a4a73d56,in Seed - 24karat Inc.,investment,,,,2023-03-16,2023-03-16,eb1aac1c-dfd2-ff0f-4ad7-28108d9b0e74,Seed - 24karat Inc.,314b3d39-9fdb-6fdd-4bbe-56d4e9a0805a,,organization,False


## Test Investment Data

In [20]:
assert (investments_df['created_at'] == '').sum() == 0, 'missing created_at value'
assert investments_df['uuid'].nunique() == len(investments_df), 'It seems like the uuid is not unique across the table'


## Organization.csv

In [21]:
for date_col in ['Total Procurement Calculation Date', 'Valuation calculation round implementation date', 'Founded Date', 'Last Funding Date', 'registration date', 'update date', 'IPO date']:
       company_df[date_col] = company_df[date_col].apply(lambda x: datetime.strptime(x, "%Y/%m/%d").strftime("%Y-%m-%d") if x else '')

In [37]:
org_cols = ['uuid', 'name', 'type', 'permalink', 'cb_url', 'rank', 'created_at',
       'updated_at', 'legal_name', 'roles', 'domain', 'homepage_url',
       'country_code', 'state_code', 'region', 'city', 'address',
       'postal_code', 'status', 'short_description', 'category_list',
       'category_groups_list', 'num_funding_rounds', 'total_funding_usd',
       'total_funding', 'total_funding_currency_code', 'founded_on',
       'last_funding_on', 'closed_on', 'employee_count', 'email', 'phone',
       'facebook_url', 'linkedin_url', 'twitter_url', 'logo_url', 'alias1',
       'alias2', 'alias3', 'primary_role', 'num_exits']

organization_df = company_df.copy()
organization_df['uuid'] = organization_df['Company url'].apply(lambda x: generate_cb_uuid("organization", x))
organization_df.rename(columns={'Company name': 'name'}, inplace=True)
organization_df['type'] = "organization"
organization_df.rename(columns={'Company url': 'permalink'}, inplace=True)
organization_df['cb_url'] = ''
organization_df['rank'] = ''
organization_df.rename(columns={'registration date': 'created_at'}, inplace=True)
organization_df.rename(columns={'update date': 'updated_at'}, inplace=True)

organization_df['legal_name'] = ''
# organization_df['roles'] Done before concat
organization_df['domain'] = organization_df['website'].apply(get_base_domain).fillna('')
organization_df.rename(columns={'website': 'homepage_url'}, inplace=True)
organization_df['country_code'] = "JP"
organization_df['state_code'] = ''
organization_df.rename(columns={'Prefectures': 'region'}, inplace=True)
organization_df['city'] = ''
# organization_df['address'] = organization_df['address']
organization_df['postal_code'] = ''
organization_df['status'] = organization_df['Investigation status'].map(map_company_status)

organization_df.rename(columns={'Description': 'short_description'}, inplace=True)
organization_df['category_list'] = [[]] * len(organization_df) # TODO. ask Conrad and Kalle to see if these twos are necessary, if necessary, generated by tags and industry
organization_df['category_groups_list'] = [[]] * len(organization_df)
organization_df['num_funding_rounds'] = organization_df['uuid'].map(cb_rounds_df.groupby('org_uuid').size())
organization_df['total_funding'] = organization_df['uuid'].map(cb_rounds_df.groupby('org_uuid')['raised_amount'].sum())
organization_df['total_funding_currency_code'] = 'JPY'
organization_df['total_funding_usd'] = organization_df['uuid'].map(cb_rounds_df.groupby('org_uuid')['raised_amount_usd'].sum())
organization_df['founded_on'] = 'Founded Date'
organization_df.rename(columns={'Founded Date': 'founded_on'}, inplace=True)
organization_df.rename(columns={'Last Funding Date': 'last_funding_on'}, inplace=True)
organization_df['closed_on'] = ''
organization_df.rename(columns={'number of employees': 'employee_count'}, inplace=True)
organization_df['email'] = ''
organization_df.rename(columns={'phone number': 'phone'}, inplace=True)
organization_df['facebook_url'] = ''
organization_df['linkedin_url'] = ''
organization_df['twitter_url'] = ''
organization_df['logo_url'] = ''
organization_df['alias1'] = ''
organization_df['alias2'] = ''
organization_df['alias3'] = ''
# organization_df['primary_role'] Done before concat
organization_df['num_exits'] = ''

cb_organization_df = organization_df[org_cols]

## Test Organization.csv

In [38]:
assert cb_organization_df['uuid'].nunique() == len(cb_organization_df), 'It seems like the uuid is not unique across the table'
assert (cb_organization_df['created_at'] == '').sum() == 0, 'missing created_at value'

# Cleanup data without domain
assert (cb_organization_df['domain']=='').sum() == 0, 'cb_organization_df has companies who missed domain value'


AssertionError: missing domain value

## Get IPO csv

In [25]:
ipo_cols = ['uuid', 'name', 'type', 'permalink', 'cb_url', 'rank', 'created_at',
       'updated_at', 'org_uuid', 'org_name', 'org_cb_url', 'country_code',
       'state_code', 'region', 'city', 'stock_exchange_symbol', 'stock_symbol',
       'went_public_on', 'share_price_usd', 'share_price',
       'share_price_currency_code', 'valuation_price_usd', 'valuation_price',
       'valuation_price_currency_code', 'money_raised_usd', 'money_raised',
       'money_raised_currency_code']

ipo_df = organization_df[organization_df['status'] == 'ipo'].copy()
ipo_df.rename(columns={'uuid': 'org_uuid'}, inplace=True)
ipo_df.rename(columns={'name': 'org_name'}, inplace=True)
ipo_df.rename(columns={'cb_url': 'org_cb_url'}, inplace=True)

ipo_df['uuid'] = ipo_df.apply(lambda row: generate_cb_uuid("ipo", row['permalink'] + row['IPO date']), axis=1)
ipo_df['name'] = ''
ipo_df['type'] = 'ipo'
ipo_df['permalink'] = ''
ipo_df['cb_url'] = ''
ipo_df['rank'] = ''
ipo_df.rename(columns={'registration date': 'created_at'}, inplace=True)
ipo_df.rename(columns={'update date': 'updated_at'}, inplace=True)
# country_code, state_code, region, city already processed and exists

ipo_df['stock_exchange_symbol'] = ''
ipo_df['stock_symbol'] = ''
ipo_df.rename(columns={'IPO date': 'went_public_on'}, inplace=True)
ipo_df['share_price_usd'] = ''
ipo_df['share_price'] = ''
ipo_df['share_price_currency_code'] = 'JPY'
ipo_df['valuation_price'] = ipo_df['Market capitalization at the time of IPO (initial price).(One million yen)'].apply((lambda x: float(x.replace(",", "")) * 1e6 if x else 0))
ipo_df['valuation_price_currency_code'] = 'JPY'
ipo_df['valuation_price_usd'] = ipo_df.apply(lambda row: get_usd(row['valuation_price_currency_code'], row['went_public_on'], row['valuation_price']), axis=1)
ipo_df['money_raised_usd'] = ''
ipo_df['money_raised'] = ''
ipo_df['money_raised_currency_code'] = 'JPY'

cb_ipo_df = ipo_df[ipo_cols]

# Get Acquisition csv

In [26]:
acq_cols = ['uuid', 'name', 'type', 'permalink', 'cb_url', 'rank', 'created_at',
       'updated_at', 'acquiree_uuid', 'acquiree_name', 'acquiree_cb_url',
       'acquiree_country_code', 'acquiree_state_code', 'acquiree_region',
       'acquiree_city', 'acquirer_uuid', 'acquirer_name', 'acquirer_cb_url',
       'acquirer_country_code', 'acquirer_state_code', 'acquirer_region',
       'acquirer_city', 'acquisition_type', 'acquired_on', 'price_usd',
       'price', 'price_currency_code']

##

# Check data is properly joined

In [42]:
cb_investments_df.head()

,uuid,name,type,permalink,cb_url,rank,created_at,updated_at,funding_round_uuid,funding_round_name,investor_uuid,investor_name,investor_type,is_lead_investor
0,58dfd788-d2fe-aac7-ac75-18992bb1257a,"in - Wellmo Co., Ltd.",investment,,,,2023-04-01,2023-04-01,03d051a6-788a-61f9-1708-735ecc40145f,"- Wellmo Co., Ltd.",314b3d39-9fdb-6fdd-4bbe-56d4e9a0805a,,organization,False
1,f0ee5328-a066-9a89-1e6d-944f351e4cff,"FUNDINNO in - Yura Holdings Co., Ltd.",investment,,,,2023-03-24,2023-03-24,2169d092-8212-9682-e18d-8597acb8b9fe,"- Yura Holdings Co., Ltd.",b0d971d8-d590-fe9e-182b-87cd7bd3cfa3,FUNDINNO,organization,True
2,4e483c58-6bc7-9c7a-19d4-8ea074fcec8c,"East Ventures in - Editor Camp Co., Ltd.",investment,,,,2023-03-17,2023-03-17,5d84e5cf-f340-4096-09fa-88cb577ee249,"- Editor Camp Co., Ltd.",0394c337-55bb-7fe7-84a4-e8c4d8ea9c03,East Ventures,organization,True
3,3e63c965-cb82-8595-4ebb-980ca8e18e90,"Angel Investor in - Editor Camp Co., Ltd.",investment,,,,2023-03-17,2023-03-17,5d84e5cf-f340-4096-09fa-88cb577ee249,"- Editor Camp Co., Ltd.",bf8165d6-fb91-2994-c281-88a4b49eed90,Angel Investor,organization,False
4,4f435a4a-c1b9-1d4a-87ce-98a0a4a73d56,in Seed - 24karat Inc.,investment,,,,2023-03-16,2023-03-16,eb1aac1c-dfd2-ff0f-4ad7-28108d9b0e74,Seed - 24karat Inc.,314b3d39-9fdb-6fdd-4bbe-56d4e9a0805a,,organization,False


In [41]:
org_uuids = cb_organization_df['uuid']

print("Remove rows in rounds_df where its org_uuid is not in org_uuids")
print(f"Remove {len(cb_rounds_df) - cb_rounds_df['org_uuid'].isin(org_uuids).sum()} rows of funding rounds data...")
cb_rounds_df = cb_rounds_df[cb_rounds_df['org_uuid'].isin(org_uuids)]
assert cb_rounds_df['org_uuid'].isin(org_uuids).all(), 'There are org_uuids in rounds_df that are not in org_uuids'

print("Remove rows in rounds_df, investments_df, ipo_df where its org_uuid is not in org_uuids")
print(f"Remove {len(cb_investments_df) - cb_investments_df['org_uuid'].isin(org_uuids).sum()} rows of investments data...")
cb_investments_df = cb_investments_df[cb_investments_df['org_uuid'].isin(org_uuids)]

assert cb_investments_df['investor_uuid'].isin(org_uuids).all(), 'There are org_uuids in investments_df that are not in org_uuids'
assert cb_ipo_df['org_uuid'].isin(org_uuids).all(), 'There are org_uuids in ipo_df that are not in org_uuids'

# Check acquirer_uuid
# Check acquiree_uuid

Remove rows in rounds_df where its org_uuid is not in org_uuids
Remove 1210 rows of funding rounds data...
Remove rows in rounds_df, investments_df, ipo_df where its org_uuid is not in org_uuids


KeyError: 'org_uuid'

In [32]:
organization_df[organization_df['name'] == 'Editor Camp Co., Ltd.']

,name,short_description,tag,INITIAL series,status,status update date,Assignee,memo,Representative's name,Total Funding Amount.(thousand yen),Total Procurement Calculation Date,Post-Money Valuation.(thousand yen),Valuation calculation round implementation date,region,founded_on,last_funding_on,type,created_at,updated_at,Investigation status,Market capitalization at the time of IPO (initial price).(One million yen),IPO date,IPO classification,employee_count,Shareholder status,origin,Universities and research institutes,Industry,market,address,Corporate number,phone,homepage_url,permalink,roles,primary_role,uuid,cb_url,rank,legal_name,domain,country_code,state_code,city,postal_code,category_list,category_groups_list,num_funding_rounds,total_funding,total_funding_currency_code,total_funding_usd,founded_on,closed_on,email,facebook_url,linkedin_url,twitter_url,logo_url,alias1,alias2,alias3,num_exits


In [29]:
rounds_df[~cb_rounds_df['org_uuid'].isin(org_uuids)]

,announced_on,org_name,INITIAL series,Funding Amount.(thousand yen),"Pre-Money Valuation.(1,000 yen)",Post-Money Valuation .(thousand yen),Investors,permalink,Company Url,uuid,name,type,cb_url,rank,created_at,updated_at,country_code,state_code,region,city,investment_type,raised_amount,raised_amount_currency_code,raised_amount_usd,post_money_valuation,post_money_valuation_currency_code,post_money_valuation_usd,investor_count,org_uuid,lead_investor_uuids
2,2023-03-17,"Editor Camp Co., Ltd.",,"70,000",,,"[{'name': 'East Ventures', 'url': '/investors/...",/companies/A-45806/funding_rounds/A-45806-01,/companies/A-45806,5d84e5cf-f340-4096-09fa-88cb577ee249,"- Editor Camp Co., Ltd.",funding_round,,,2023-03-17,2023-03-17,JP,,NaN,,,70000000.0,JPY,531020.00,0.000000e+00,JPY,0.000000e+00,2,5041776a-5d01-e702-1a8b-23e6f3b72780,0394c337-55bb-7fe7-84a4-e8c4d8ea9c03
23,2023-03-10,"Thermalytica Co., Ltd.",,"112,500",,,"[{'name': 'KSPInc.', 'url': '/investors/89'}, ...",/companies/A-40567/funding_rounds/A-40567-02,/companies/A-40567,852077d5-27dd-b3b2-a88c-fe819eede914,"- Thermalytica Co., Ltd.",funding_round,,,2023-03-10,2023-03-10,JP,,NaN,,,112500000.0,JPY,834300.00,0.000000e+00,JPY,0.000000e+00,3,0110c85c-5442-b6ab-10e8-8833d27fe860,5a8f7221-c36c-74df-1e5e-3f468cd7f95e
45,2023-03-03,"DentaLight Co., Ltd.",Series A,"32,855","2,656,885","2,689,740",[],/companies/A-28972/funding_rounds/A-28972-25,/companies/A-28972,29d47e8a-63bd-c8be-590a-2c4462967ba9,"Series A - DentaLight Co., Ltd.",funding_round,,,2023-03-03,2023-03-03,JP,,NaN,,series_a,32855000.0,JPY,241878.51,2.656885e+09,JPY,1.955999e+07,0,a8cc20f5-bf1f-60e5-5a6c-a92a0cd0499a,
63,2023-02-28,"NOT A HOTEL Co., Ltd.",Series A,"369,095","12,395,192","12,764,287",[],/companies/A-36874/funding_rounds/A-36874-28,/companies/A-36874,05134ecf-6b58-8127-8542-6745f3ade680,"Series A - NOT A HOTEL Co., Ltd.",funding_round,,,2023-02-28,2023-02-28,JP,,NaN,,series_a,369095000.0,JPY,2711371.87,1.239519e+10,JPY,9.105508e+07,0,ac23ebda-11ae-b84f-8e43-0ca3bb527ac3,
65,2023-02-28,"Axis Route Holdings Co., Ltd.",,,,,"[{'name': 'Bewith Co., Ltd.', 'url': '/investo...",/companies/A-04830/funding_rounds/A-04830-06,/companies/A-04830,c011c362-57b3-492a-9fd6-f778b6898264,"- Axis Route Holdings Co., Ltd.",funding_round,,,2023-02-28,2023-02-28,JP,,NaN,,,0.0,JPY,0.00,0.000000e+00,JPY,0.000000e+00,1,b72614a3-6e37-f9fe-5ebb-21ac617f775c,3c29fdc1-b469-07b8-c76c-be5574b8e3dc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56908,1961-06-21,"Links Co., Ltd.",,,,,[],/companies/A-03347/funding_rounds/A-03347-00,/companies/A-03347,d500b757-8283-2ebc-5eb2-d59eb2a47d23,"- Links Co., Ltd.",funding_round,,,1961-06-21,1961-06-21,JP,,NaN,,,0.0,JPY,0.00,0.000000e+00,JPY,0.000000e+00,0,6333ae5c-f88a-d937-a509-71a090613262,
56931,1960-05-01,"Sanbo Co., Ltd.",,"32,000",,,[],/companies/A-03351/funding_rounds/A-03351-00,/companies/A-03351,e8e62d34-c707-6503-4c97-f0321680ece4,"- Sanbo Co., Ltd.",funding_round,,,1960-05-01,1960-05-01,JP,,NaN,,,32000000.0,JPY,277216.00,0.000000e+00,JPY,0.000000e+00,0,8f6bf908-36ac-3bfc-be64-b25cde62cb35,
56985,1957-06-08,"Japan Mechatronics Co., Ltd.",,"1,000",,,[],/companies/A-03270/funding_rounds/A-03270-00,/companies/A-03270,508d18bc-fbcf-6146-884d-1e051edec1cc,"- Japan Mechatronics Co., Ltd.",funding_round,,,1957-06-08,1957-06-08,JP,,NaN,,,1000000.0,JPY,8663.00,0.000000e+00,JPY,0.000000e+00,0,2c9172ed-322e-1d9a-15fe-ca70e5e9e152,
57021,1954-04-01,"Asano Ironworks Co., Ltd.",,,,,[],/companies/A-03855/funding_rounds/A-03855-00,/companies/A-03855,92664d27-77f8-09c2-d020-bcab9abc85ae,"- Asano Ironworks Co., Ltd.",funding_round,,,1954-04-01,1954-04-01,JP,,NaN,,,0.0,JPY,0.00,0.000000e+00,JPY,0.000000e+00,0,9aefbf52-bb52-5efc-ef4e-166774be244f,


In [ ]:
cb_rounds_df[~cb_rounds_df['org_uuid'].isin(org_uuids)]